<a href="https://colab.research.google.com/github/Elbereth-Elentari/Book_recommender/blob/master/baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB

from sklearn.metrics import recall_score

from IPython.display import clear_output

In [2]:
def read_input(jl_file):
    catalogue_df = pd.read_json(jl_file, lines=True, orient='records').fillna('')
    catalogue_df = catalogue_df[catalogue_df['interesting'] != '']
    catalogue_df.replace({'interesting':'yes'}, 1, inplace=True)
    catalogue_df.replace({'interesting':'no'}, 0, inplace=True)

    def join_tokens(row):
        return ' '.join(row['tokens'])

    catalogue_df['tokens'] = catalogue_df.apply(join_tokens, axis=1)
    return catalogue_df

In [14]:
def create_datasets(catalogue_df):
    X = catalogue_df[['year', 'pages', 'tokens']]
    y = catalogue_df['interesting']
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

    scaler = MinMaxScaler().fit(X_train[['year', 'pages']])
    X_train_scaled = scaler.transform(X_train[['year', 'pages']])
    X_test_scaled = scaler.transform(X_test[['year', 'pages']])

    cvect = CountVectorizer()
    X_train_counts = cvect.fit_transform(X_train['tokens'])
    X_test_counts = cvect.transform(X_test['tokens'])
    tfidf = TfidfTransformer(use_idf=False)
    X_train_transformed = tfidf.fit_transform(X_train_counts)
    X_test_transformed = tfidf.transform(X_test_counts)

    X_train_array = X_train_transformed.toarray()
    X_test_array = X_test_transformed.toarray()
    X_train_final = np.hstack((X_train_array, X_train_scaled))
    X_test_final = np.hstack((X_test_array, X_test_scaled))

    return X_train_final, X_test_final, y_train, y_test

In [29]:
def create_model_dictionary():
    models = {'KNN':
              {'classifier':KNeighborsClassifier(),
               'grid_values':{'n_neighbors':[2, 3, 4, 5, 6]}},
              'logistic regression':
              {'classifier':LogisticRegression(max_iter=4000),
               'grid_values':{'C':[20, 30, 40, 600, 700, 800]}},
              'SVC linear':
              {'classifier':SVC(kernel='linear'),
               'grid_values':{'C':[1.5, 2, 2.5, 3, 3.5]}},
              'SVC rbf':
              {'classifier':SVC(kernel='rbf'),
               'grid_values':{'C':[3.5, 4, 4.5, 6.5, 7, 7.5]}},
              'Naive Bayes multinomial':
              {'classifier':MultinomialNB(),
               'grid_values':{'alpha':[0.015, 0.02, 0.025, 0.03, 0.035]}},
              'Naive Bayes Gaussian':
              {'classifier':GaussianNB(),
               'grid_values':{'var_smoothing':[0.035, 0.04, 0.045, 0.15, 0.2, 0.25]}}}
    return models

In [30]:
def calculate_baselines():
    baselines_df = pd.DataFrame(columns=['param_name', 'en', 'param_en', 'pl', 'param_pl'])
    models = create_model_dictionary()
    catalogue = read_input('/content/drive/My Drive/Library_catalogue_preprocessed.jl')

    for language in ['en', 'pl']:
        print(f'Processing {language}')
        lang_catalogue = catalogue[catalogue['language'] == language]
        X_train, X_test, y_train, y_test = create_datasets(lang_catalogue)

        for model in models:
            print(f'Calculating {model}')
            grid_values = models[model]['grid_values']
            grid_search = GridSearchCV(models[model]['classifier'], param_grid=grid_values, scoring='recall')
            grid_search.fit(X_train, y_train)
            recall = recall_score(y_test, grid_search.predict(X_test))
            baselines_df.loc[model, language] = recall
            param_name = list(models[model]['grid_values'].keys())[0]
            baselines_df.loc[model, 'param_name'] = param_name
            baselines_df.loc[model, f'param_{language}'] = grid_search.best_params_[param_name]

    clear_output()
    print(f'Mean recall scores')
    return baselines_df

calculate_baselines()

Mean recall scores


,param_name,en,param_en,pl,param_pl
KNN,n_neighbors,0.526316,3,0.410526,3
logistic regression,C,0.421053,20,0.505263,700
SVC linear,C,0.526316,2,0.526316,3.5
SVC rbf,C,0.421053,3.5,0.568421,7
Naive Bayes multinomial,alpha,0.473684,0.025,0.431579,0.02
Naive Bayes Gaussian,var_smoothing,0.894737,0.15,0.8,0.035
